In [33]:
from decoder import Decoder
from decoderattn import Attention_Decoder
from encoder import Encoder
from evaluatemodel import MySeq2SeqModelEvaluation
from train_with_batch import Seq2SeqTrainer
from trainmodel import Seq2SeqTrainer
from data_proc import DataPreparation
from data_proc import *


In [18]:
import torch
import random
from io import open
import unicodedata
import string
import re
import random
import pickle
import numpy as np
import math
from mosestokenizer import *
from collections import Counter

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [37]:
SOS_token = 0
EOS_token = 1
max_length = 50


In [9]:
obj= DataPreparation('fra', 'eng', True)
input_lang, output_lang, pairs= obj.prepare_data(reverse=True)


Reading lines...
Form:  ['l or a . dollars l once ?', ' gold ?']
Read 2000 sentence pairs
Trimmed to 1837 sentence pairs
Counting words...
Counted words:
eng 6924
fra 6019


In [26]:
hidden_size = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
attn_decoder = Attention_Decoder(hidden_size, output_lang.n_words, max_length , dropoutP=0.1).to(device)



#encoder1 = EncoderRNN(input_dim = input_lang.n_words,emb_dim =256,enc_hid_dim = 1000, dec_hid_dim = 1000, dropout = 0.6) 
encoder = Encoder(input_size = input_lang.n_words, hidden_size = hidden_size, device = device).to(device)



In [47]:
import torch
import numpy as np
import time
import torch.optim as optim
import torch.nn as nn
import random
from data_proc import *

from decoderattn import Attention_Decoder
from decoder import Decoder


class Seq2SeqTrainer:
    def __init__(self, encoder_model, decoder_model, max_sequence_length, start_token, end_token, input_lang ,output_lang , device):
        self.encoder_model = encoder_model
        #self.decoder_type = decoder_type
        self.max_sequence_length = max_sequence_length
        self.start_token = start_token
        self.end_token = end_token
        self.device = device
        self.input_lang = input_lang
        self.output_lang = output_lang
        self.decoder_model= decoder_model

        '''
        if self.decoder_type == 'simple':
            self.decoder_model = Decoder(encoder_model.hidden_size * 2, decoder_model.output_size)
        elif self.decoder_type == 'attention':
            self.decoder_model = Attention_Decoder(encoder_model.hidden_size, decoder_model.output_size, max_sequence_length)
        else:
            raise ValueError("Invalid decoder type. Supported types are 'simple' and 'attention'.")'''

    def train(self, input_sequence, target_sequence, encoder_optimizer, decoder_optimizer, loss_criterion, probability=0.5):
        # Initializations
        input_len = input_sequence.size(0)
        target_len = target_sequence.size(0)
        total_loss = 0
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        encoder_hidden_states = torch.zeros(self.max_sequence_length, self.encoder_model.hidden_size * 2, device=self.device)

        # Initialize the decoder
        current_input = torch.tensor([[self.start_token]], device=self.device)

        # Encoding Phase
        hidden_state = self.encoder_model.init_hidden()

        for i in range(input_len):
            encoder_output, hidden_state = self.encoder_model(input_sequence[i])
            encoder_hidden_states[i] = encoder_output[0, 0]

            # Decoder takes the last hidden state of the encoder
            hidden_state_decoder = hidden_state

            # Decoding Phase
            for j in range(target_len):
                decoder_output, hidden_state_decoder, _ = self.decoder_model(current_input, hidden_state_decoder, encoder_hidden_states)
                total_loss += loss_criterion(decoder_output, target_sequence[j])

                explore_decision = (np.random.rand() < probability)
                if explore_decision:
                    current_input = target_sequence[j]
                else:
                    topv, topi = decoder_output.topk(1)
                    current_input = topi.squeeze().detach()

                if current_input.item() == self.end_token:
                    break

        # Backpropagation and weight update
        total_loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        return total_loss.item() / target_len

    def custom_training_function(self, num_epochs, print_interval=500, pairs=0, plot_interval=50, learning_rate=0.01):
        # Initialize loss tracking variables
        all_losses = []
        current_loss_sum = 0
        plot_loss_sum = 0

        # Initialize Adam optimizers for the two networks with the specified learning rate
        optimizer_encoder = optim.Adam(self.encoder_model.parameters(), lr=learning_rate)
        optimizer_decoder = optim.Adam(self.decoder_model.parameters(), lr=learning_rate)

        # Generate training data using random pairs and define the loss criterion
        training_data = [[tensorsFromPair(random.choice(pairs) , self.input_lang,self.output_lang,self.device)] for _ in range(num_epochs)]

        for epoch in range(1, num_epochs + 1):
            data_point = training_data[epoch - 1]
            input_data = data_point[0]
            target_data = data_point[1]

            loss = self.train(input_data, target_data, optimizer_encoder, optimizer_decoder, nn.NLLLoss())
            current_loss_sum += loss
            plot_loss_sum += loss

            # Print the average loss at specified intervals
            if epoch % print_interval == 0:
                avg_print_loss = current_loss_sum / print_interval
                current_loss_sum = 0
                print('%s (%d %d%%) %.4f' % (self.calculate_time_elapsed(time.time(), epoch / num_epochs),
                                             epoch, epoch / num_epochs * 100, avg_print_loss))

            # Track losses for plotting at specified intervals
            if epoch % plot_interval == 0:
                avg_plot_loss = plot_loss_sum / plot_interval
                all_losses.append(avg_plot_loss)
                plot_loss_sum = 0

        # Print the list of losses for visualization
        print("Loss Visualization: ", all_losses)

    # Function to calculate elapsed and remaining time
    def calculate_time_elapsed(self, start, progress):
        elapsed_seconds = time.time() - start
        remaining_seconds = (elapsed_seconds / progress) * (1 - progress)
        return f'Time Elapsed: {int(elapsed_seconds)}s, Remaining: {int(remaining_seconds)}s'

In [48]:

import unicodedata
import re
import torch

SOS_token = 0
EOS_token = 1

MAX_LENGTH = 50


def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence, device):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair,input_lang,output_lang, device):
    input_tensor = tensorFromSentence(input_lang, pair[0],device)
    target_tensor = tensorFromSentence(output_lang, pair[1],device)
    return (input_tensor, target_tensor)

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


class DataPreparation:
    def __init__(self, lang1, lang2, reverse=False):
        #self.input_lang, self.output_lang, self.pairs = self.prepare_data(reverse)
        self.lang1=lang1
        self.lang2=lang2
        self.reverse=reverse

    def unicode_to_ascii(self, s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

    def normalize_string(self, s):
        s = self.unicode_to_ascii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        return s

    def read_langs(self, lang1, lang2, reverse=False):
        print("Reading lines...")

        # Read the file and split into lines
        lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

        # Split every line into pairs and normalize
        pairs = [[self.normalize_string(s) for s in l.split('\t')] for l in lines]

        # Reverse pairs, make Lang instances
        if reverse:
            pairs = [list(reversed(p)) for p in pairs]
            input_lang = Lang(lang2)
            output_lang = Lang(lang1)
        else:
            input_lang = Lang(lang1)
            output_lang = Lang(lang2)

        return input_lang, output_lang, pairs

    def filter_pair(self, p):
        return len(p[0].split(' ')) < MAX_LENGTH and \
               len(p[1].split(' ')) < MAX_LENGTH

    def filter_pairs(self, pairs):
        return [pair for pair in pairs if self.filter_pair(pair)]

    def prepare_data(self, reverse=True):
        self.input_lang, self.output_lang, pairs = self.read_langs(self.lang1, self.lang2, reverse)
        print("Form: ", pairs[0])
        print("Read %s sentence pairs" % len(pairs))
        pairs = self.filter_pairs(pairs)
        print("Trimmed to %s sentence pairs" % len(pairs))
        print("Counting words...")
        for pair in pairs:
            self.input_lang.add_sentence(pair[0])
            self.output_lang.add_sentence(pair[1])
        print("Counted words:")
        print(self.input_lang.name, self.input_lang.n_words)
        print(self.output_lang.name, self.output_lang.n_words)
        return self.input_lang, self.output_lang, pairs



In [49]:
start_token = 0
end_token = 1
max_length = 50
num_epochs = 1
TRAIN1=Seq2SeqTrainer(encoder, attn_decoder, max_length, start_token, end_token,input_lang ,output_lang , device)
custom_train1=TRAIN1.custom_training_function(num_epochs, 500, pairs, plot_interval=50, learning_rate=0.01)

IndexError: list index out of range